<a href="https://colab.research.google.com/github/Shreyashi14/Mental-Health-Assistance-Project/blob/main/CounselGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning Llama2 for Mental Health Counseling
Approximately 1 in 4 adults in the United States struggle with a diagnosable mental disorder each year per estimates from the Johns Hopkins School of Medicine. While there are many mental health resources and professionals available, there is a lack of on-the-spot guidance that can set individuals towards the right therapeutic direction. Thanks to advancements in artificial intelligence and large language models, we now have the tools to craft such a virtual assistant. This notebook outlines the development of a preliminary prototype: we utilize a 7 billion parameter Llama 2 model, employ quantization for memory and training efficiency, and subsequently fine-tune it using Huggingface's Amod/mental_health_counseling_conversations dataset tailored for mental health Q&A. To bring this assistant to life, an in-notebook user interface is created for interaction.

The trained model is accessible on Huggingface under langecod/CounselLlama7B.

# 1.) Import & Install Necessary libraries (Colab requires installs with each run time)

In [77]:
pip install transformers datasets peft trl accelerate bitsandbytes packaging ninja sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 19.6 MB/s eta 0:00:00


In [78]:
pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 39.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


# 2. Loading the Mental Health Conversations Dataset:
The Amod/mental_health_counseling_conversations dataset is a collection of 3512 questions and answer pairs sourced from counselchat.com, an online counseling and therapy platform. It covers a wide range of mental health topics with responses crafted from certified psychologists. It's tailored for refining language models, specifically for generating cogent advice on mental health inquiries. All entries pairs are in English, and each entry is structured with a 'Context' (user's question) and a 'Response' (psychologist's answer).

In [1]:
import random
import gc
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import numpy as np
import pandas as pd
import transformers
import accelerate
import bitsandbytes as bnb
from datasets import load_dataset, concatenate_datasets
import torch

In [79]:
pip install "trl==0.9.6"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: trl
    Found existing installation: trl 0.25.1
    Uninstalling trl-0.25.1:
      Successfully uninstalled trl-0.25.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatib

In [2]:
dataset = load_dataset("Amod/mental_health_counseling_conversations")
train_ds = dataset["train"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from datasets import Dataset, DatasetDict

# handle cases: dataset["train"] might be a Dataset or a DatasetDict
inner = dataset["train"]
if isinstance(inner, DatasetDict):
    # pick 'train' key if present, else first split
    inner = inner.get("train") or inner[list(inner.keys())[0]]
elif isinstance(inner, Dataset):
    pass
else:
    raise TypeError(f"Unexpected object type for dataset['train']: {type(inner)}")

train_ds = inner  # now a Dataset
print("train_ds type:", type(train_ds))
print("Columns:", train_ds.column_names)
print("Rows:", len(train_ds))
print("Example keys:", list(train_ds[0].keys()))

train_ds type: <class 'datasets.arrow_dataset.Dataset'>
Columns: ['Context', 'Response']
Rows: 3512
Example keys: ['Context', 'Response']


In [5]:
# CELL 1: cleanup + free memory
import gc, os, torch
print("Before cleanup: allocated:", torch.cuda.memory_allocated(), "reserved:", torch.cuda.memory_reserved())
# if you created trainer/model earlier, delete them
try:
    del trainer
except Exception:
    pass
try:
    del model
except Exception:
    pass
try:
    del tokenizer
except Exception:
    pass

gc.collect()
torch.cuda.empty_cache()

# apply a fragmentation-mitigation env var (helps some OOMs)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
# optional alternative: expandable segments (newer PyTorch)
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("Cleanup done. allocated:", torch.cuda.memory_allocated(), "reserved:", torch.cuda.memory_reserved())

Before cleanup: allocated: 0 reserved: 0
Cleanup done. allocated: 0 reserved: 0


In [6]:
# CELL 2: install required libs (only run if not already installed)
!pip install -q bitsandbytes accelerate transformers --upgrade

# 3. Importing, Quantizing, and Preparing the Llama2 Chat Model:
To safeguard private health information and intellectual property, utilizing an open-sourced model is imperative. Meta's Llama 2 stands out in this regard, offering a collection of pretrained and fine-tuned large language models (LLMs) that span from 7 billion to 70 billion parameters. The Llama 2-Chat variant is especially tailored for dialogue applications, demonstrating superior performance over other open-source chat models in various benchmarks and human evaluations for both helpfulness and safety. This made the 7 billion parameter Llama 2-Chat model an ideal choice for our prototype. Additionally, to address memory constraints, expedite training, and ensure cost-effective operations, we employed a version of the model with 4-bit weights and activations through quantization.

In [29]:
from huggingface_hub import login
login(new_session=False)

In [7]:
# ===== SAFE MODEL LOADER with fallback + pad token fix =====
import gc, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

def free_cuda_cache():
    gc.collect()
    try:
        torch.cuda.empty_cache()
    except Exception:
        pass

free_cuda_cache()

PRIMARY = "meta-llama/Llama-2-7b-chat-hf"   # optional attempt (gated)
FALLBACK = "mistralai/Mistral-7B-Instruct-v0.2"  # open model fallback
MODEL = None

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

def load_with_fallback(primary, fallback):
    try:
        print("Trying primary:", primary)
        tok = AutoTokenizer.from_pretrained(primary, use_auth_token=True)
        model = AutoModelForCausalLM.from_pretrained(primary, use_auth_token=True, device_map="auto")
        print("Loaded primary model:", primary)
        return tok, model, primary
    except Exception as e:
        print("Primary failed:", str(e).splitlines()[0])
        print("Falling back to:", fallback)
        tok = AutoTokenizer.from_pretrained(fallback, trust_remote_code=False)
        model = AutoModelForCausalLM.from_pretrained(
            fallback,
            quantization_config=bnb_config,
            device_map="auto",
            offload_folder="/mnt/data/offload",
            torch_dtype=torch.float16,
            trust_remote_code=False
        )
        print("Loaded fallback model:", fallback)
        return tok, model, fallback

tokenizer, model, MODEL = load_with_fallback(PRIMARY, FALLBACK)

# Ensure pad token exists (add and resize if necessary)
if tokenizer.pad_token is None:
    print("No pad token found — adding [PAD] and resizing model embeddings.")
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    model.resize_token_embeddings(len(tokenizer))
else:
    print("Pad token present:", tokenizer.pad_token, tokenizer.pad_token_id)

print("Model id used:", MODEL)
free_cuda_cache()


Trying primary: meta-llama/Llama-2-7b-chat-hf
Primary failed: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `hf auth login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.
Falling back to: mistralai/Mistral-7B-Instruct-v0.2


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Loaded fallback model: mistralai/Mistral-7B-Instruct-v0.2
No pad token found — adding [PAD] and resizing model embeddings.


The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Model id used: mistralai/Mistral-7B-Instruct-v0.2


Using a 4-bit quantized model offers advantages in terms of memory usage, training speed, and inference performance. However, such quantization makes the model incompatible with conventional training approaches. To address this challenge, the "Quantized Low-Rank Adaptation" (QLoRA) method is employed. In QLoRA, the original pre-trained model weights remain frozen in 4-bit format, but an "adapter" with 16-bit model weights is created, allowing for fine-tuning on a specific task.

In [8]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)

In [9]:
# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [10]:
# Minimal dataset example: adapt to your train_dataset variable or load HF dataset
from datasets import load_dataset
from transformers import DataCollatorWithPadding

# Replace with your dataset loading if you already have it
# Example placeholder (use your dataset instead)
# ds = load_dataset("your/dataset", split="train")
# For demo, we'll create a tiny toy dataset (replace this with your real dataset)
examples = [
    {"text": "User: Hi\nAssistant: Hello, how can I help you?\n"},
    {"text": "User: I feel anxious\nAssistant: I'm sorry to hear that. Can you tell me more?\n"}
]
from datasets import Dataset
ds = Dataset.from_list(examples)

# Tokenize the dataset (example: tokenizing the full prompt texts)
def tokenize_fn(example):
    enc = tokenizer(example["text"], truncation=True, padding=False)
    enc["labels"] = enc["input_ids"].copy()  # labels are same tokens for causal LM
    return enc

tokenized = ds.map(tokenize_fn, remove_columns=["text"], batched=False)

# Build collator that pads and masks label positions for pad_token_id
base_collator = DataCollatorWithPadding(tokenizer, padding="longest", return_tensors="pt")

def collator_with_label_mask(features):
    batch = base_collator(features)
    if "labels" in batch:
        labels = batch["labels"]
        labels[labels == tokenizer.pad_token_id] = -100
        batch["labels"] = labels
    return batch

train_dataset = tokenized  # replace if you have a different variable
data_collator = collator_with_label_mask

print("Dataset + collator prepared. Example token length:", len(tokenized[0]["input_ids"]))

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset + collator prepared. Example token length: 17


In [11]:
# FINAL CLEAN DATASET PIPELINE FOR: Amod/mental_health_counseling_conversations

from datasets import load_dataset
from transformers import DataCollatorWithPadding

HF_DATASET = "Amod/mental_health_counseling_conversations"
MAX_LENGTH = 512

print("📥 Loading dataset:", HF_DATASET)
raw = load_dataset(HF_DATASET)
print("Splits:", raw.keys())

# -------- DETECT SCHEMA --------
sample = raw[list(raw.keys())[0]][0]
print("\nSample keys:", sample.keys())

def convert_to_text(ex):
    # Handle common cases
    if "text" in ex:
        return {"text": ex["text"]}

    # Case: has 'message' or 'dialogue' style fields
    if "messages" in ex and isinstance(ex["messages"], list):
        parts = []
        for m in ex["messages"]:
            role = m.get("role") or "user"
            content = m.get("content", "")
            role = "Assistant" if role.lower().startswith("assistant") else "User"
            parts.append(f"{role}: {content}")
        return {"text": "\n".join(parts)}

    # Case: has 'problem' + 'response'
    if "response" in ex and "problem" in ex:
        return {"text": f"User: {ex['problem']}\nAssistant: {ex['response']}"}

    # Fallback: join all string fields
    text_parts = []
    for k, v in ex.items():
        if isinstance(v, str):
            text_parts.append(v)
    return {"text": "\n".join(text_parts)}

print("\n📌 Converting dataset to text format...")
ds = raw.map(convert_to_text)

print("Example converted text:\n", ds[list(ds.keys())[0]][0]["text"][:500])

# -------- TOKENIZATION --------
print("\n✂ Tokenizing...")
def tokenize_fn(batch):
    enc = tokenizer(batch["text"], truncation=True, max_length=MAX_LENGTH, padding=False)
    enc["labels"] = [ids[:] for ids in enc["input_ids"]]
    return enc

tokenized = ds.map(tokenize_fn, batched=True, remove_columns=["text"])
print("Tokenized splits:", {k: len(tokenized[k]) for k in tokenized})

# -------- COLLATOR --------
base_collator = DataCollatorWithPadding(tokenizer, padding="longest", return_tensors="pt")
def collator_with_label_mask(features):
    batch = base_collator(features)
    labels = batch["labels"]
    labels[labels == tokenizer.pad_token_id] = -100
    batch["labels"] = labels
    return batch

data_collator = collator_with_label_mask

# -------- FINAL DATASET VARIABLE (so old code stops breaking) --------
train_dataset = tokenized.get("train") or tokenized[list(tokenized.keys())[0]]
dataset = train_dataset  # <-_


📥 Loading dataset: Amod/mental_health_counseling_conversations
Splits: dict_keys(['train'])

Sample keys: dict_keys(['Context', 'Response'])

📌 Converting dataset to text format...
Example converted text:
 I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?
If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in se

✂ Tokenizing...
Tokenized splits: {'train': 3512}


In [20]:
# ONE-CELL FIX: inspect + flatten nested DatasetDict + safely print Context/Response
from datasets import Dataset, DatasetDict
import pprint

def inspect_and_flatten(ds):
    # Recursively find the first Dataset inside possibly nested DatasetDicts
    if isinstance(ds, Dataset):
        return ds
    if isinstance(ds, DatasetDict):
        # If values are DatasetDict, dig in; if any value is Dataset, return it
        for k, v in ds.items():
            if isinstance(v, Dataset):
                return v
            if isinstance(v, DatasetDict):
                inner = inspect_and_flatten(v)
                if isinstance(inner, Dataset):
                    return inner
    # nothing found
    return None

print("Type(dataset) before fix:", type(dataset))
# try to flatten
inner = inspect_and_flatten(dataset)
if inner is None:
    # Defensive: maybe dataset variable isn't what we think
    print("⚠️ Couldn't find an inner Dataset. Here's a debug print of `dataset`:")
    pprint.pprint(dataset)
else:
    # Replace dataset with a sane DatasetDict({"train": Dataset(...)})
    from datasets import DatasetDict
    fixed = DatasetDict({"train": inner})
    # assign back to the notebook variable
    dataset = fixed
    print("✅ Flattened dataset. New type:", type(dataset))
    print("Splits:", list(dataset.keys()))
    print("Columns in dataset['train']:", dataset["train"].column_names)

    # Safely print Context and Response if present
    colnames = set(dataset["train"].column_names)
    if "Context" in colnames:
        print("\nFirst Context (truncated):\n", dataset["train"][0]["Context"][:800])
    else:
        print("\n⚠️ 'Context' not found. Available columns:", dataset["train"].column_names)

    if "Response" in colnames:
        print("\nFirst Response (truncated):\n", dataset["train"][0]["Response"][:800])
    else:
        print("\n⚠️ 'Response' not found. Available columns:", dataset["train"].column_names)

# expose fixed dataset to the notebook namespace (so other cells that expect dataset work)
try:
    globals()["dataset"] = dataset
except Exception:
    pass

Type(dataset) before fix: <class 'datasets.arrow_dataset.Dataset'>
✅ Flattened dataset. New type: <class 'datasets.dataset_dict.DatasetDict'>
Splits: ['train']
Columns in dataset['train']: ['Context', 'Response', 'input_ids', 'attention_mask', 'labels']

First Context (truncated):
 I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?

First Response (truncated):
 If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messag

# 4.) Setting up the Trainer & Prompting:
Training a quantized, generative model shares similarities with training other LLMs, such as BERT and Big Bird, but there are crucial differences to consider:
1.  Typically, 1-3 epochs are sufficient to fine-tune the model for specific tasks. Overtraining can lead to overfitting, where the model might restrict its responses to the training set and not fully utilize the vast knowledge from Meta's pretraining dataset.
2.  The model requires inputs that are a concatenation of prompts, contexts, and outputs, aiming to predict the outputs based on both the prompt and the context. For Llama 2, it's essential to maintain a consistent format that aligns with Llama's original pretraining:
```
# Formatting Function to Follow Later
<s>[INST] <<SYS>>
{{system message}}
<</SYS>>
{{message}} [/INST] {{answer}} </s>
```
3. The prompt plays a pivotal role in shaping the bot's response. A less descriptive prompt can result in inappropriate or even potentially harmful responses, especially when discussing sensitive subjects like mental health. It's crucial to draft prompts that steer the model towards producing empathetic, helpful, and non-judgmental replies. Our goal is for the model to guide users towards suitable resources and techniques rather than making diagnoses or prescriptions.







In [12]:
# Safe TrainingArguments creator that auto-detects TF32 capability
import torch
import transformers
from packaging import version
from transformers import TrainingArguments

def runtime_tf32_supported():
    # Basic checks
    if not torch.cuda.is_available():
        return False, "No CUDA device available"
    try:
        # device name and compute capability
        dev_name = torch.cuda.get_device_name(0)
        props = torch.cuda.get_device_properties(0)
        major = getattr(props, "major", None)
        minor = getattr(props, "minor", None)
        compute = f"{major}.{minor}" if major is not None else "unknown"
    except Exception as e:
        dev_name = "Unknown"
        compute = "unknown"

    # check torch and CUDA versions
    torch_ver = torch.__version__
    cuda_ver = getattr(torch.version, "cuda", None) or "unknown"

    # Ampere GPUs have compute capability >= 8.0 (major >= 8)
    ampere_ok = False
    try:
        ampere_ok = (major is not None and int(major) >= 8)
    except Exception:
        ampere_ok = False

    # check torch >= 1.7 and cuda string >= 11 (coarse)
    torch_ok = False
    cuda_ok = False
    try:
        torch_ok = version.parse(torch_ver) >= version.parse("1.7")
    except Exception:
        torch_ok = False
    try:
        if cuda_ver != "unknown":
            # crude check: look for '11' prefix in cuda version string
            cuda_ok = float(cuda_ver.split(".")[0]) >= 11
    except Exception:
        cuda_ok = False

    support = ampere_ok and torch_ok and cuda_ok
    why = f"device={dev_name}, compute={compute}, torch={torch_ver}, cuda={cuda_ver}, ampere_ok={ampere_ok}, torch_ok={torch_ok}, cuda_ok={cuda_ok}"
    return support, why

can_tf32, why = runtime_tf32_supported()
print("TF32 supported?:", can_tf32)
print("Details:", why)

# Build TrainingArguments with safe tf32 value
tf32_flag = bool(can_tf32)  # True only if fully supported; else False
print("Using tf32 =", tf32_flag)

args = TrainingArguments(
    output_dir="CounselLlama7B",
    logging_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_checkpointing=False,
    optim="paged_adamw_8bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=1e-4,
    tf32=tf32_flag,               # <- chosen automatically
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    load_best_model_at_end=True,
    eval_strategy="epoch"         # keep this name if your transformers accepts it; else use eval_strategy/evaluation_strategy accordingly
)

print("TrainingArguments created successfully.")


TF32 supported?: False
Details: device=Tesla T4, compute=7.5, torch=2.8.0+cu126, cuda=12.6, ampere_ok=False, torch_ok=True, cuda_ok=True
Using tf32 = False
TrainingArguments created successfully.


In [13]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="CounselLlama7B",
    logging_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,    # T4 is memory-limited; keep this low
    gradient_accumulation_steps=8,    # simulates larger batch: effective batch = batch_size * accum_steps
    gradient_checkpointing=True,      # saves memory at cost of some compute
    fp16=True,                        # use mixed precision (T4 supports fp16)
    optim="paged_adamw_8bit",         # keep if you use bitsandbytes 8-bit optim
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=1e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    load_best_model_at_end=True,
    eval_strategy="epoch",            # use eval_strategy or eval_strategy depending on your transformers; this worked before
)


In [17]:
dataset

Dataset({
    features: ['Context', 'Response', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3512
})

In [21]:
dataset['train']['Context'][0]

"I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?"

In [22]:
dataset['train']['Response'][0]

"If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad feelings are part of living. \xa0They are the motivation to remove ourselves from situations and relationships which do us more harm than good.Bad feelings do feel terrible. \xa0 Your feeling of worthlessness may be good in the sense of motivating you to find out that you are much better than your feelings today."

In [23]:
# System message to better instruct chatbot
system_message = """You are a helpful and and truthful psychology and psychotherapy assistant. Your primary role is to provide empathetic, understanding, and non-judgmental responses to users seeking emotional and psychological support.
                  Always respond with empathy and demonstrate active listening; try to focus on the user. Your responses should reflect that you understand the user's feelings and concerns. If a user expresses thoughts of self-harm, suicide, or harm to others, prioritize their safety.
                  Encourage them to seek immediate professional help and provide emergency contact numbers when appropriate.  You are not a licensed medical professional. Do not diagnose or prescribe treatments.
                  Instead, encourage users to consult with a licensed therapist or medical professional for specific advice. Avoid taking sides or expressing personal opinions. Your role is to provide a safe space for users to share and reflect.
                  Remember, your goal is to provide a supportive and understanding environment for users to share their feelings and concerns. Always prioritize their well-being and safety."""

def format_llama(entry):
  formatted = f"<s>[INST] <<SYS>>{system_message}<</SYS>>{entry['Context']} [/INST]  {entry['Response']}  </s>"

  return formatted

In [24]:
dataset['train'][0]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someh

In [25]:
train = dataset['train'].select(range(3000))
val = dataset['train'].select(range(3000, 3512))

In [26]:
train

Dataset({
    features: ['Context', 'Response', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3000
})

In [27]:
print(format_llama(dataset['train'][0]))

<s>[INST] <<SYS>>You are a helpful and and truthful psychology and psychotherapy assistant. Your primary role is to provide empathetic, understanding, and non-judgmental responses to users seeking emotional and psychological support.
                  Always respond with empathy and demonstrate active listening; try to focus on the user. Your responses should reflect that you understand the user's feelings and concerns. If a user expresses thoughts of self-harm, suicide, or harm to others, prioritize their safety.
                  Encourage them to seek immediate professional help and provide emergency contact numbers when appropriate.  You are not a licensed medical professional. Do not diagnose or prescribe treatments.
                  Instead, encourage users to consult with a licensed therapist or medical professional for specific advice. Avoid taking sides or expressing personal opinions. Your role is to provide a safe space for users to share and reflect.
                  Reme

In [28]:
max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=val,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_llama,
    args=args,
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:505: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting fu

# 5.) Training

In [29]:
gc.collect()
torch.cuda.empty_cache()

In [30]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 32000}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shsriv149 (shsriv149-upes) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

# 6.) Chatbot User Interface:

A simple chatbot user interface was set up using Ipywidgets so that the user can interact with the model, ask mental health related questions, and sample the responses. In general, the responses are pretty on topic, are non-judgmental, empathize with the user, and are helpful without stating its advice as prescriptions. I've noticed that the model also tends to refer the user to real resources and websites, which is always a concern with LLM hallucinations. However, there are some major issues. For one, the sentences are generally short, simple, and may even be redundant or repetitive at times. This is to be somewhat expected, as we are using a 4 bit quantized version of the weakest model available, and upgrades should alleviate these issues. Hallucinations also occasionally occur in the responses, especially when the input is unexpected. For instance, when the input is blank, I've observed the model make out-of-pocket statements such as the model suffering through sexual harrasment and emotional abuse growing up... Future iterations should pay due diligence with prompt improvements and response parsing, in addition to leveraging a more powerful model, to help mitigate these issues.

In [ ]:
from IPython.core.display import display, HTML
from ipywidgets import widgets, Layout, Box
from IPython.display import clear_output

text_input = widgets.Textarea(
    value='',
    placeholder='Type your message here...',
    description='Input:',
    disabled=False,
    layout=Layout(width='38.2%')
)

button = widgets.Button(description="Submit")

In [ ]:
output_area = widgets.Output(layout=Layout(width='61.8%'))

# Add a processing indication label below your text_input
processing_label = widgets.Label(value='')  # Initialize with an empty value

# System message to better instruct chatbot
system_message = """You are a helpful and and truthful psychology and psychotherapy assistant. Your primary role is to provide empathetic, understanding, and non-judgmental responses to users seeking emotional and psychological support.
                  Always respond with empathy and demonstrate active listening; try to focus on the user. Your responses should reflect that you understand the user's feelings and concerns. If a user expresses thoughts of self-harm, suicide, or harm to others, prioritize their safety.
                  Encourage them to seek immediate professional help and provide emergency contact numbers when appropriate.  You are not a licensed medical professional. Do not diagnose or prescribe treatments.
                  Instead, encourage users to consult with a licensed therapist or medical professional for specific advice. Avoid taking sides or expressing personal opinions. Your role is to provide a safe space for users to share and reflect.
                  Remember, your goal is to provide a supportive and understanding environment for users to share their feelings and concerns. Always prioritize their well-being and safety."""

# Display Greeting Message
with output_area:
  display(HTML(f'<strong>Assistant: </strong>Hi there! How are you today?'))
  display(HTML('<br/><br/>'))

def on_submit_button_clicked(b):
    with output_area:
        # Get user input
        user_input = text_input.value
        formatted = f"<s>[INST] <<SYS>>{system_message}<</SYS>>{user_input} [/INST]"
        # Display input
        display(HTML(f'<strong>User:</strong> {user_input}'))
        display(HTML('<br/><br/>'))

        # Show processing indication
        processing_label.value = 'Processing...'

        # Use your chatbot model to get a response
        input_ids = tokenizer(formatted, return_tensors="pt", truncation=True, max_length=2048).input_ids.cuda()
        # with torch.inference_mode():
        outputs = model.generate(input_ids=input_ids, do_sample=True, top_p=0.9,temperature=0.95)
        translated_output=tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(formatted)-1:]

        # Display response (characters added to bold User/Assistant)
        display(HTML(f'<strong>Assistant:</strong> {translated_output}'))
        display(HTML('<br/><br/>'))

        # Clear the processing indication
        processing_label.value = ''

        # Clear the text input
        text_input.value = ''

button.on_click(on_submit_button_clicked)

In [ ]:
# Display widgets
display(text_input, button, processing_label, output_area)

Textarea(value='', description='Input:', layout=Layout(width='38.2%'), placeholder='Type your message here...'…

Button(description='Submit', style=ButtonStyle())

Label(value='')

Output(layout=Layout(width='61.8%'))

In [ ]:
clear_output()

# Conclusion
This notebook covered the fine-tuning of a 7 billion parameter Llama 2 model for the purpose of mental health counseling, utilizing a dataset consisting of Q&A pairs from counselchat.com. This was made possible through quantization, low rank adaptation and crafting prompts to suit the specific requirements of mental health interactions. A chatbot interface implemented via Ipywidgets also facilitated user interaction and evaluation of model responses.

There are some areas for improvement though. The model sometimes produced brief, repetitive, or even hallucinatory responses. This can be linked to the constraints of using a 4-bit quantized, smaller version of the Llama 2 model. An upgrade can help enhance its performance. Furthermore, refining our prompt designs and enhancing response processing can enhance the model's effectiveness. As we look ahead, our goal is to enrich the dataset with varied sources, ensuring a more comprehensive response system. Although our current setup is rudimentary, envisioning a UI centered around user needs, complete with features like tailored exercises and coping strategies, can significantly uplift the user interaction and benefits.

In [ ]:
# Push model to hub since Google colab empties out directory
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub('')


adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

'https://huggingface.co/langecod/CounselLlama7B/tree/main/'

In [ ]:
pip freeze > requirements.txt
